# AWS Budgeting

## Configuring Settings for AWS Budgeting

To set up an AWS budget for PODPAC resources, set the `AWS_BUDGET_AMOUNT` and `AWS_BUDGET_EMAIL` settings:

In [ ]:
import podpac
from podpac import settings

# configure budget settings
settings["AWS_BUDGET_AMOUNT"] = 10.0             # budget for all podpac AWS resources, in USD
settings["AWS_BUDGET_EMAIL"] = "test@podpac.org" # notification e-mail for budget alarm at 80% of Budget amount

## Build AWS Node with budget

PODPAC AWS manager will use `settings` as defaults when building/updating AWS resources.

In [ ]:
from podpac.managers import aws

# create a Lambda Node (uses `settings` as defaults)
node = aws.Lambda()

# build function and all resources
# This can take up to 15-20 seconds if all resources need to be created.
# Note: if AWS resources already exist, POPDAC use these resources as is and will skip the generation of these resources.
node.build()

# view details of built resources
node.describe()

## Budget Tracking

- Resource tracking is performed using the special tag `_podpac_resource_hash`.
This allows PODPAC to apply budgets specific Nodes independently.
- See budget console: https://console.aws.amazon.com/billing/home?#/budgets/details?name=podpac-test-smap-13-budget
- Budgets are updated **every 24 hours**, so it is still possible to over-spend in a single 24 hour period.

In [ ]:
aws.create_budget()

## Run PODPAC Lambda Node

### SinCoords Example

This example wraps a `SinCoords` node, executes it on our Lambda function, and returns the output `UnitsDataArray`.

In [ ]:
import podpac
from podpac import clinspace
from podpac.managers import aws

In [ ]:
sin_coords_node = podpac.algorithm.SinCoords()
coordinates = podpac.Coordinates([clinspace(-90, 90, 180), clinspace(90,-90, 180), '2018-01-01'], ['lat', 'lon', 'time'])

In [ ]:
node =  podpac.managers.aws.Lambda(source=sin_coords_node)
output = node.eval(coordinates)
output

In [ ]:
output.plot()